# Run Updates

### Connect to Graph

In [1]:
import py2neo
graph = py2neo.Graph(user='neo4j',password='admin')

# Global Root Update

In [3]:
# Count the number of relationships
query_attributenfathers='''
MATCH (n:DunsID)
OPTIONAL MATCH (n)-[r:isChildOf]->(c)
WITH n, count(c) AS nfathers
SET n.nfathers = nfathers
'''
a=graph.run(query_attributenfathers)
print(a.stats())
#if there are no relationships then update the isglobalroot attribute
query_attributeglobalroot='''
MATCH (n:DunsID)
WITH n, (CASE n.nfathers WHEN 0 THEN True Else False END) as root
SET n.isglobalroot = root
'''
a=graph.run(query_attributeglobalroot)
print(a.stats())

#Update the labels accordingly
query_labelglobalroot1='''
MATCH (n:DunsID{isglobalroot:False})
REMOVE n:GlobalRoot, n.rootname'''
a=graph.run(query_labelglobalroot1)
print(a.stats())

query_labelglobalroot2='''
MATCH (m:DunsID{isglobalroot:True})
Set m:GlobalRoot,m.rootname = m.companyname;
'''
a=graph.run(query_labelglobalroot2)
print(a.stats())

{'constraints_added': 0, 'constraints_removed': 0, 'indexes_added': 0, 'indexes_removed': 0, 'labels_added': 0, 'labels_removed': 0, 'nodes_created': 0, 'nodes_deleted': 0, 'properties_set': 238405, 'relationships_deleted': 0, 'relationships_created': 0, 'contains_updates': True}
{'constraints_added': 0, 'constraints_removed': 0, 'indexes_added': 0, 'indexes_removed': 0, 'labels_added': 0, 'labels_removed': 0, 'nodes_created': 0, 'nodes_deleted': 0, 'properties_set': 238405, 'relationships_deleted': 0, 'relationships_created': 0, 'contains_updates': True}
{'constraints_added': 0, 'constraints_removed': 0, 'indexes_added': 0, 'indexes_removed': 0, 'labels_added': 0, 'labels_removed': 0, 'nodes_created': 0, 'nodes_deleted': 0, 'properties_set': 0, 'relationships_deleted': 0, 'relationships_created': 0, 'contains_updates': False}
{'constraints_added': 0, 'constraints_removed': 0, 'indexes_added': 0, 'indexes_removed': 0, 'labels_added': 170780, 'labels_removed': 0, 'nodes_created': 0, 'no

### Global Root and Direct Parent attribute update for export

#### Duns5

In [4]:
query_duns5='''
MATCH (n:DunsID)-[:isChildOf*0..]->(r:GlobalRoot)
SET n.duns5_calc=r.dunsid;
'''
a=graph.run(query_duns5)
print(a.stats())

{'constraints_added': 0, 'constraints_removed': 0, 'indexes_added': 0, 'indexes_removed': 0, 'labels_added': 0, 'labels_removed': 0, 'nodes_created': 0, 'nodes_deleted': 0, 'properties_set': 238405, 'relationships_deleted': 0, 'relationships_created': 0, 'contains_updates': True}


#### Parent Duns

In [5]:
query_duns3='''
MATCH (n:DunsID{isglobalroot:False})-[:isChildOf]->(r:DunsID)
SET n.parentdunsid=r.dunsid;
'''
a=graph.run(query_duns3)
print(a.stats())
query_duns3_bis='''
MATCH (n:GlobalRoot)
SET n.parentdunsid=n.dunsid;'''
a=graph.run(query_duns3_bis)
print(a.stats())

{'constraints_added': 0, 'constraints_removed': 0, 'indexes_added': 0, 'indexes_removed': 0, 'labels_added': 0, 'labels_removed': 0, 'nodes_created': 0, 'nodes_deleted': 0, 'properties_set': 67625, 'relationships_deleted': 0, 'relationships_created': 0, 'contains_updates': True}
{'constraints_added': 0, 'constraints_removed': 0, 'indexes_added': 0, 'indexes_removed': 0, 'labels_added': 0, 'labels_removed': 0, 'nodes_created': 0, 'nodes_deleted': 0, 'properties_set': 170780, 'relationships_deleted': 0, 'relationships_created': 0, 'contains_updates': True}


# Spent

### Per Duns Node

In [6]:
query_nodeturnover='''
MATCH(m:DunsID)
SET m.nodeturnover=0
with count(m) as temp
MATCH (n:DunsID)<-[:hasDuns]-(s:ErpID)
WITH n,SUM(s.turnover) as erpturnover
SET n.nodeturnover=erpturnover;
'''
a = graph.run(query_nodeturnover)
print(a.stats())

{'constraints_added': 0, 'constraints_removed': 0, 'indexes_added': 0, 'indexes_removed': 0, 'labels_added': 0, 'labels_removed': 0, 'nodes_created': 0, 'nodes_deleted': 0, 'properties_set': 279610, 'relationships_deleted': 0, 'relationships_created': 0, 'contains_updates': True}


In [7]:
query_nodeordervolume='''
MATCH(m:DunsID)
SET m.nodeordervolume=0
with count(m) as temp
MATCH (n:DunsID)<-[:hasDuns]-(s:ErpID)
WITH n,SUM(s.ordervolume) as erpordervolume
SET n.nodeordervolume=erpordervolume;
'''
a = graph.run(query_nodeordervolume)
print(a.stats())

{'constraints_added': 0, 'constraints_removed': 0, 'indexes_added': 0, 'indexes_removed': 0, 'labels_added': 0, 'labels_removed': 0, 'nodes_created': 0, 'nodes_deleted': 0, 'properties_set': 279610, 'relationships_deleted': 0, 'relationships_created': 0, 'contains_updates': True}


### Aggregate turnover

In [8]:
old_query='''
MATCH (n:DunsID)<-[:isChildOf*0..]-(child:DunsID)
WITH n,sum(child.nodeturnover) AS value
SET n.aggregateturnover = value
'''

query_leaves='''
MATCH path=(r:GlobalRoot)<-[:isChildOf*0..]-(b:DunsID), (b)<-[:isChildOf]-(l:DunsID)
WITH b, length(path) as depth, collect(l) as ls
ORDER BY depth desc
SET b.aggregateturnover = REDUCE(acc = COALESCE(b.nodeturnover,0), l IN ls | acc + COALESCE(l.aggregateturnover, l.nodeturnover, 0))
'''
a=graph.run(query_leaves)
print(a.stats())

query_root='''
MATCH (n:GlobalRoot)
OPTIONAL MATCH (n)<-[:isChildOf]-(child:DunsID)
WITH n,sum(child.aggregateturnover) AS value
SET n.aggregateturnover = value+n.nodeturnover
'''
a=graph.run(query_root)
print(a.stats())

{'constraints_added': 0, 'constraints_removed': 0, 'indexes_added': 0, 'indexes_removed': 0, 'labels_added': 0, 'labels_removed': 0, 'nodes_created': 0, 'nodes_deleted': 0, 'properties_set': 44258, 'relationships_deleted': 0, 'relationships_created': 0, 'contains_updates': True}
{'constraints_added': 0, 'constraints_removed': 0, 'indexes_added': 0, 'indexes_removed': 0, 'labels_added': 0, 'labels_removed': 0, 'nodes_created': 0, 'nodes_deleted': 0, 'properties_set': 170780, 'relationships_deleted': 0, 'relationships_created': 0, 'contains_updates': True}


### Aggregate ordervolume

In [9]:
query_leaves='''
MATCH path=(r:GlobalRoot)<-[:isChildOf*0..]-(b:DunsID), (b)<-[:isChildOf]-(l:DunsID)
WITH b, length(path) as depth, collect(l) as ls
ORDER BY depth desc
SET b.aggregateordervolume = REDUCE(acc = COALESCE(b.nodeordervolume,0), l IN ls | acc + COALESCE(l.aggregateordervolume, l.nodeordervolume, 0))
'''
a=graph.run(query_leaves)
print(a.stats())

query_root='''
MATCH (n:GlobalRoot)
OPTIONAL MATCH (n)<-[:isChildOf]-(child:DunsID)
WITH n,sum(child.aggregateordervolume) AS value
SET n.aggregateordervolume = value+n.nodeordervolume
'''
a=graph.run(query_root)
print(a.stats())

{'constraints_added': 0, 'constraints_removed': 0, 'indexes_added': 0, 'indexes_removed': 0, 'labels_added': 0, 'labels_removed': 0, 'nodes_created': 0, 'nodes_deleted': 0, 'properties_set': 44258, 'relationships_deleted': 0, 'relationships_created': 0, 'contains_updates': True}
{'constraints_added': 0, 'constraints_removed': 0, 'indexes_added': 0, 'indexes_removed': 0, 'labels_added': 0, 'labels_removed': 0, 'nodes_created': 0, 'nodes_deleted': 0, 'properties_set': 170780, 'relationships_deleted': 0, 'relationships_created': 0, 'contains_updates': True}
